# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4, timeoutcontroller
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1616403584.0
Main triggered 1616403584.5
Main triggered 1616403585.0
Print triggered 1616403585.0
Main triggered 1616403585.5


# Controller Base Class (eFMU)

In [4]:
    # Test controller
    testcontroller = testcontroller1() #this controller multiplies the inputs
    # Get all variables
    variables = testcontroller.get_model_variables()
    # Makeup some inputs
    inputs = {}
    for var in variables:
        inputs[var] = random.randint(1,50)
    # Query controller
    print ('Log-message', testcontroller.do_step(inputs=inputs))
    print ('Input', testcontroller.input)
    print ('Output', testcontroller.output)
    print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 44, 'b': 41}
Output {'c': 1804}
Output {'c': 1804}


# Controller Stack Class (single-thread/multi-thread)

In [5]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control for 20 seconds.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 00:59:47.724858112,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 00:59:52.326619904,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 00:59:56.956931072,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:00:01.571141888,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:00:06.194205952,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 00:59:47.724858112,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 00:59:52.326619904,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 00:59:56.956931072,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:00:01.571141888,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:00:06.194205952,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 00:59:47.724858112,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 00:59:52.326619904,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 00:59:56.956931072,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:00:01.571141888,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:00:06.194205952,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 00:59:47.724858112,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 00:59:52.326619904,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 00:59:56.956931072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:00:01.571141888,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:00:06.194205952,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 00:59:47.724858112,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 00:59:52.326619904,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 00:59:56.956931072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:00:01.571141888,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:00:06.194205952,30.0,4.0,120.0,testcontroller1 did a computation!


25.23270297050476


## Multi Thread

In [7]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, workers=200, timeout=2)


# Call query_control for 2 minutes. We should expect close to 120 recordings per "first-in-queue" controller.
# In multi thread mod, each call of query_control will trigger a computation for all controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
ctrl_stack.run_query_control_for(120)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:00:15.034287104,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:00:16.066661888,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:00:17.115322112,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:00:18.119165952,10.0,4.0,40.0,testcontroller1 did a computation!
...,...,...,...,...
2021-03-22 01:02:10.223262976,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:02:11.231052032,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:02:12.247048960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:02:13.258017024,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:00:15.034287104,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:00:16.066661888,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:00:17.115322112,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:00:18.119165952,40.0,10.0,400.0,testcontroller2 did a computation!
...,...,...,...,...
2021-03-22 01:02:10.223262976,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:02:11.231052032,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:02:12.247048960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:02:13.258017024,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:00:15.034287104,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:00:17.115322112,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:00:19.125078016,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:00:21.166226944,400.0,40.0,16000.0,testcontroller3 did a computation!
...,...,...,...,...
2021-03-22 01:02:06.143130112,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:02:08.186873088,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:02:10.223262976,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:02:12.247048960,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:00:15.034287104,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:00:16.066661888,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:00:17.115322112,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:00:19.125078016,20.0,4.0,80.0,testcontroller3 did a computation!
...,...,...,...,...
2021-03-22 01:02:07.145739008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:02:08.186873088,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:02:10.223262976,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:02:12.247048960,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:00:15.034287104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:00:16.066661888,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:00:17.115322112,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:00:18.119165952,30.0,4.0,120.0,testcontroller1 did a computation!
...,...,...,...,...
2021-03-22 01:02:10.223262976,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:02:11.231052032,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:02:12.247048960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:02:13.258017024,30.0,4.0,120.0,testcontroller1 did a computation!


122.52529692649841


In [10]:
# refresh the mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

#Adding timeoutcontroller
controllers['timeout'] = {'fun': timeoutcontroller, 'sampletime': 5}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2, workers=200)
print("done initializing")

# Call query_control for 20 seconds. We should expect a bunch of timeouts.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

done initializing
Controller timeout: timeout


/Users/deoxi/FMLC/FMLC/stackedclasses.py:296: Warning: Controller timeout timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout: timeout
Controller timeout: timeout
Controller timeout: timeout


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:04:09.905000960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:10.953433088,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:11.960603904,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:13.009150976,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:14.018461952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:15.028592128,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:16.054961920,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:17.066618112,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-22 01:04:18.076888064,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:04:09.905000960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:10.953433088,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:11.960603904,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:13.009150976,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:14.018461952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:15.028592128,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:16.054961920,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:17.066618112,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-22 01:04:18.076888064,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:04:09.905000960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:10.953433088,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:13.009150976,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:14.018461952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:16.054961920,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:18.076888064,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:19.105029120,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:21.148752896,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-22 01:04:22.172626944,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:04:09.905000960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:10.953433088,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:13.009150976,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:15.028592128,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:17.066618112,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:19.105029120,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:21.148752896,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:22.172626944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-22 01:04:24.227887104,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-22 00:59:43.613923072,NaN,NaN,NaN,Initialize
2021-03-22 01:04:09.905000960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:10.953433088,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:11.960603904,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:13.009150976,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:14.018461952,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:15.028592128,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:16.054961920,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:17.066618112,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-22 01:04:18.076888064,30.0,4.0,120.0,testcontroller1 did a computation!


,a,Logging
2021-03-22 00:59:43.613923072,NaN,Initialize
2021-03-22 01:04:09.905000960,seen due to late computation,Compute ok
2021-03-22 01:04:14.922281984,seen due to late computation,Compute ok
2021-03-22 01:04:19.956827904,seen due to late computation,Compute ok
2021-03-22 01:04:24.990424064,seen due to late computation,Compute ok


22.285892724990845
